[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/chain.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58238466-lesson-4-chain)

# Chain

## Review

We built a simple graph with nodes, normal edges, and conditional edges.

## Goals

Now, let's build up to a simple chain that combines 4 [concepts](https://python.langchain.com/v0.2/docs/concepts/):

* Using [chat messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as our graph state
* Using [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) in graph nodes
* [Binding tools](https://python.langchain.com/v0.2/docs/concepts/#tools) to our chat model
* [Executing tool calls](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling) in graph nodes 

![Screenshot 2024-08-21 at 9.24.03 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dd607b08df5e1101_chain1.png)

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation. 

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`. 

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call. 

Let's create a list of messages. 

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author 
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [10]:
import {AIMessage, HumanMessage} from "@langchain/core/messages";

let messages = [new AIMessage({content:"So you said you were researching ocean mammals?", name:"Model"})]
messages.push(new HumanMessage({content:"Yes, that's right.",name:"Lance"}))
messages.push(new AIMessage({content:"Great, what would you like to learn about.", name:"Model"}))
messages.push(new HumanMessage({content:"I want to learn about the best place to see Orcas in the US.", name:"Lance"}))

messages.forEach(message => {
    console.log(message);
  });

SyntaxError: Identifier 'messages' has already been declared

## Chat Models

[Chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) can use a sequence of message as input and support message types, as discussed above.

There are [many](https://python.langchain.com/v0.2/docs/concepts/#chat-models) to choose from! Let's work with OpenAI. 

We can load a chat model and invoke it with out list of messages.

We can see that the result is an `AIMessage` with specific `response_metadata`.

In [9]:
import { ChatOpenAI } from "@langchain/openai";
const llm = new ChatOpenAI({model:"gpt-4o"});
const result = await llm.invoke(messages);
console.log('result :>> ', result.content);

TypeError: promptValue.toChatMessages is not a function

In [5]:
console.log(result.response_metadata)

{
  tokenUsage: { promptTokens: 67, completionTokens: 299, totalTokens: 366 },
  finish_reason: "stop",
  usage: {
    prompt_tokens: 67,
    completion_tokens: 299,
    total_tokens: 366,
    prompt_tokens_details: { cached_tokens: 0, audio_tokens: 0 },
    completion_tokens_details: {
      reasoning_tokens: 0,
      audio_tokens: 0,
      accepted_prediction_tokens: 0,
      rejected_prediction_tokens: 0
    }
  },
  system_fingerprint: "fp_831e067d82"
}


## Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language. 

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user. 

And, it will return an output that adheres to the tool's schema. 

[Many LLM providers support tool calling](https://python.langchain.com/v0.1/docs/integrations/chat/) and [tool calling interface](https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/) in LangChain is simple. 
 
You can simply pass any Python `function` into `ChatModel.bind_tools(function)`.

![Screenshot 2024-08-19 at 7.46.28 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dc1c17a7a57f9960_chain2.png)

Let's showcase a simple example of tool calling!
 
The `multiply` function is our tool.

In [12]:
/**
 * Multiply a and b.
 * 
 * @param a - first number
 * @param b - second number
 */
function multiply(a: number, b: number): number {
    return a * b;
  }

const llm_with_tools = llm.bindTools([multiply])

If we pass an input - e.g., `"What is 2 multiplied by 3"` - we see a tool call returned. 

The tool call has specific arguments that match the input schema of our function along with the name of the function to call.

```
{'arguments': '{"a":2,"b":3}', 'name': 'multiply'}
```

In [11]:
const tool_call = await llm_with_tools.invoke([new HumanMessage({content:"What is 2 multiplied by 3", name:"Lance"})])
console.log('tool_call :>> ', tool_call);

Error: 400 Invalid type for 'tools[0]': expected an object, but got null instead.

: 

In [8]:
tool_call.additional_kwargs['tool_calls']

[{'id': 'call_OP4qwtGk4gTIaIySPFHkpbB2',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'},
  'type': 'function'}]

## Using messages as state

With these foundations in place, we can now use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages) in our graph state.

Let's define our state, `MessagesState`, as a `TypedDict` with a single key: `messages`.

`messages` is simply a list of messages, as we defined above (e.g., `HumanMessage`, etc).

In [19]:
import {BaseMessage, AIMessage, HumanMessage} from "@langchain/core/messages";

class MessagesState {
    messages: [BaseMessage]
}

## Reducers

Now, we have a minor problem! 

As we discussed, each node will return a new value for our state key `messages`.

But, this new value will [will override](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) the prior `messages` value.
 
As our graph runs, we want to **append** messages to to our `messages` state key.
 
We can use [reducer functions](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) address this.

Reducers allow us to specify how state updates are performed.

If no reducer function is specified, then it is assumed that updates to the key should *override it* as we saw before.
 
But, to append messages, we can use the pre-built `add_messages` reducer.

This ensures that any messages are appended to the existing list of messages.

We annotate simply need to annotate our `messages` key with the `add_messages` reducer function as metadata.

In [20]:
import type {BaseMessage, } from "@langchain/core/messages";
import { Annotation, messagesStateReducer, type Messages } from "@langchain/langgraph";

const MessageState = Annotation.Root({
    messages: Annotation<BaseMessage[], Messages>({
      reducer: messagesStateReducer,
    }),
  });

Since having a list of messages in graph state is so common, LangGraph has a pre-built [`MessagesState`](https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate)! 

`MessagesState` is defined: 

* With a pre-build single `messages` key
* This is a list of `AnyMessage` objects 
* It uses the `add_messages` reducer

We'll usually use `MessagesState` because it is less verbose than defining a custom `TypedDict`, as shown above.

In [13]:
//from langgraph.graph import MessagesState

//class MessagesState(MessagesState):
 // Add any keys needed beyond messages, which is pre-built pass

To go a bit deeper, we can see how the `add_messages` reducer works in isolation.

In [21]:
// Initial state
const initial_messages = [new AIMessage({content:"Hello! How can I assist you?", name:"Model"}),
                    new HumanMessage({content:"I'm looking for information on marine biology.", name:"Lance"})
                   ]

// New message to add
const new_message = new AIMessage({content:"Sure, I can help with that. What specifically are you interested in?", name:"Model"})

// Test
messagesStateReducer(initial_messages , new_message)

[
  AIMessage {
    "id": "78b71e1d-1339-4e2f-b3fb-b2ed02bdd313",
    "content": "Hello! How can I assist you?",
    "name": "Model",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  },
  HumanMessage {
    "id": "743c993d-cf12-45ac-b4be-ddf30190156d",
    "content": "I'm looking for information on marine biology.",
    "name": "Lance",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  AIMessage {
    "id": "fc996e9a-9b11-4714-8c0f-ce720dee6065",
    "content": "Sure, I can help with that. What specifically are you interested in?",
    "name": "Model",
    "additional_kwargs": {},
    "response_metadata": {},
    "tool_calls": [],
    "invalid_tool_calls": []
  }
]

## Our graph

Now, lets use `MessagesState` with a graph.

In [46]:
import {
    StateGraph,
    START,
    END,
    Annotation,
    MessagesAnnotation
  } from "@langchain/langgraph";
  import {AIMessage, HumanMessage} from "@langchain/core/messages";
  
  export const StateAnnotation = Annotation.Root({
    ...MessagesAnnotation.spec,
  });

// Node
const tool_calling_llm = (state: any) => {
  return {"messages": [llm_with_tools.invoke(state["messages"])]}
}

// Build graph
const builder = new StateGraph(MessagesAnnotation);
builder.addNode("tool_calling_llm", tool_calling_llm)
.addEdge(START, "tool_calling_llm")
.addEdge("tool_calling_llm", END)
export const graph = builder.compile()

// View
// display(Image(graph.get_graph().draw_mermaid_png()))

StateGraph {
  nodes: {
    tool_calling_llm: {
      runnable: RunnableCallable {
        lc_serializable: false,
        lc_kwargs: {},
        lc_runnable: true,
        name: "tool_calling_llm",
        lc_namespace: [ "langgraph" ],
        func: [Function: tool_calling_llm],
        tags: undefined,
        config: undefined,
        trace: false,
        recurse: true
      },
      retryPolicy: undefined,
      metadata: undefined,
      input: {
        messages: BinaryOperatorAggregate {
          ValueType: undefined,
          UpdateType: undefined,
          lg_is_channel: true,
          lc_graph_name: "BinaryOperatorAggregate",
          value: [],
          operator: [Function: messagesStateReducer],
          initialValueFactory: [Function: default]
        }
      },
      subgraphs: undefined
    }
  },
  edges: Set(2) {
    [ "__start__", "tool_calling_llm" ],
    [ "tool_calling_llm", "__end__" ]
  },
  branches: {},
  entryPoint: undefined,
  compiled: true,
  cha

If we pass in `Hello!`, the LLM responds without any tool calls.

In [49]:
const nextState = await graph.invoke({messages: [new HumanMessage("Hello")],})

console.log(nextState)


Error: Unable to coerce message from array: only human, AI, system, or tool message coercion is currently supported.

Received: {}

Troubleshooting URL: https://js.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE/


The LLM chooses to use a tool when it determines that the input or task requires the functionality provided by that tool.

In [52]:
const next = await graph.invoke({"messages": [new HumanMessage("Multiply 2 and 3")]})
console.log('messages :>> ', next);

Error: Unable to coerce message from array: only human, AI, system, or tool message coercion is currently supported.

Received: {}

Troubleshooting URL: https://js.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE/
